# Normative modelling for dMRI data

In [1]:
import os
os.chdir('/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/braincharts/scripts/')

import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
import sys
from random import sample

from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from nm_utils import calibration_descriptives, remove_bad_subjects, load_2d

ukb_root = '/project_freenas/3022017.02/UKB'
sys.path.append(os.path.join(ukb_root,'scripts'))
from ukb_utils import get_variables_UKB, lookup_UKB
ukb_idp_dir = os.path.join(ukb_root,'phenotypes','current')
root_dir = '/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/braincharts'

### Load data

In [2]:
# load sex
field_codes = ['eid','31-0.0']
field_names = ['eid', 'sex']
df_sex, subs = get_variables_UKB(os.path.join(ukb_idp_dir,'01_basic_demographics.csv'), field_codes, field_names)

# load age and site
field_codes = ['eid', '21003-2.0', '54-2.0']
field_names = ['eid', 'age', 'site']
df_age, subs = get_variables_UKB(os.path.join(ukb_idp_dir,'99_miscellaneous.csv'), field_codes, field_names)

# load dMRI derived phenotypes
field_codes = ['eid', '25059-2.0', '25101-2.0']
field_names = ['eid', 'FA_corpus_callosum', 'FA_uncinate_fasciculus']
df_dmri, subs = get_variables_UKB(os.path.join(ukb_idp_dir,'31_brain_IDPs.csv'), field_codes, field_names)

In [3]:
dmri_data = df_sex.join(df_age).join(df_dmri)
dmri_data.dropna(inplace=True)
dmri_data.to_csv(os.path.join(root_dir,'data_dmri.csv'))

In [4]:
# where the raw data are stored
data_dir = os.path.join(root_dir,'data')

# where the analysis takes place
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

### Split the data

In [5]:
### Load all the subject IDs that have qc data available
pickled_data = '/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/subjects_qc.pkl'
with open(pickled_data, 'rb') as f: 
    subs_qc = pickle.load(f)
### Load the subjects IDs that have manual qc labels available 
pickled_data = '/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/labeledsubs_qc.pkl'
with open(pickled_data, 'rb') as f: 
    labeled_qc = pickle.load(f)
### Subtract the labeled subs from the total subs
keys = list(labeled_qc.columns.values)
i1 = subs_qc.set_index(keys).index
i2 = labeled_qc.set_index(keys).index
unlabeled_subs = subs_qc[~i1.isin(i2)]
print('There are', len(subs_qc), 'subjects in total')
print('There are', len(unlabeled_subs), 'subjects without manually assigned labels')
### Split the subjects into training and testing
### Select 5k subjects randomly for the testing dataset
# unlabeled_subs = unlabeled_subs.subs.tolist()
# subs_test = sample(unlabeled_subs,5000)
# subs_test = pd.DataFrame(subs_test)
# subs_test.columns = ['subs']
# subs_test.to_pickle('/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/subs_test_5k.pkl')
pickled_data2 = '/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/subs_test_5k.pkl'
with open(pickled_data2, 'rb') as f: 
    subs_test = pickle.load(f)
### Create the training subs list
keys = list(subs_test.columns.values)
i1 = unlabeled_subs.set_index(keys).index
i2 = subs_test.set_index(keys).index
subs_train = unlabeled_subs[~i1.isin(i2)]
print('There are', len(subs_train), 'subjects for training')
print('There are', len(subs_test), 'subjects for testing')
# subs_train.to_pickle('/home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/subs_train_5k.pkl')

There are 23158 subjects in total
There are 22658 subjects without manually assigned labels
There are 17658 subjects for training
There are 5000 subjects for testing


In [7]:
### Create the training and testig dataset based on the subject lists
df_tr = dmri_data[dmri_data.index.isin(subs_train.subs)]
df_te = dmri_data[dmri_data.index.isin(subs_test.subs)]

In [8]:
df_te

,sex,site,age,FA_corpus_callosum,FA_uncinate_fasciculus
eid,,,,,
1001815,0.0,11025.0,59.0,0.725544,0.570983
1002690,1.0,11027.0,77.0,0.749954,0.564741
1002758,1.0,11027.0,60.0,0.706880,0.493949
1004244,0.0,11027.0,51.0,0.752798,0.519203
1004617,1.0,11027.0,71.0,0.755851,0.562250
...,...,...,...,...,...
6022149,1.0,11025.0,67.0,0.701434,0.466851
6023174,0.0,11025.0,69.0,0.681440,0.518593
6023696,1.0,11025.0,69.0,0.746134,0.556581


### Configure which models to fit

In [12]:
# with open(os.path.join(root_dir,'data','phenotypes.txt')) as f:
#     idp_ids = f.read().splitlines()
idp_ids = ['FA_corpus_callosum', 'FA_uncinate_fasciculus']
site_ids =  sorted(set(df_tr['site'].to_list()))

### Configure model parameters

In [13]:
# which data columns do we wish to use as covariates? 
cols_cov = ['age','sex']

# which warping function to use? We can set this to None in order to fit a vanilla Gaussian noise model
warp =  'WarpSinArcsinh'

# limits for cubic B-spline basis 
xmin = -5 
xmax = 110

# Do we want to force the model to be refit every time? 
force_refit = True

# Absolute Z treshold above which a sample is considered to be an outlier (without fitting any model)
outlier_thresh = 7

### Fit the model

In [14]:
for idp_num, idp in enumerate(idp_ids): 
    print('Running IDP', idp_num, idp, ':')
   
    # set output dir 
    idp_dir = os.path.join(out_dir, idp)
    os.makedirs(os.path.join(idp_dir), exist_ok=True)
    os.chdir(idp_dir)
    
    # extract the response variables for training and test set
    y_tr = df_tr[idp].to_numpy() 
    y_te = df_te[idp].to_numpy()
    
    # remove gross outliers and implausible values
#     yz_tr = (y_tr - np.mean(y_tr)) / np.std(y_tr)
#     yz_te = (y_te - np.mean(y_te)) / np.std(y_te)
#     nz_tr = np.bitwise_and(np.abs(yz_tr) < outlier_thresh, y_tr > 0)
#     nz_te = np.bitwise_and(np.abs(yz_te) < outlier_thresh, y_te > 0)
#     y_tr = y_tr[nz_tr]
#     y_te = y_te[nz_te]
    
    # write out the response variables for training and test
    resp_file_tr = os.path.join(idp_dir, 'resp_tr.txt')
    resp_file_te = os.path.join(idp_dir, 'resp_te.txt') 
    np.savetxt(resp_file_tr, y_tr)
    np.savetxt(resp_file_te, y_te)
        
    # configure the design matrix
    X_tr = create_design_matrix(df_tr[cols_cov], 
                                site_ids = df_tr['site'],
                                basis = 'bspline', 
                                xmin = xmin, 
                                xmax = xmax)
    X_te = create_design_matrix(df_te[cols_cov], 
                                site_ids = df_te['site'], 
                                all_sites=site_ids,
                                basis = 'bspline', 
                                xmin = xmin, 
                                xmax = xmax)

    # configure and save the covariates
    cov_file_tr = os.path.join(idp_dir, 'cov_bspline_tr.txt')
    cov_file_te = os.path.join(idp_dir, 'cov_bspline_te.txt')
    np.savetxt(cov_file_tr, X_tr)
    np.savetxt(cov_file_te, X_te)

    if not force_refit and os.path.exists(os.path.join(idp_dir, 'Models', 'NM_0_0_estimate.pkl')):
        print('Making predictions using a pre-existing model...')
        suffix = 'predict'
        
        # Make prdictsion with test data
        predict(cov_file_te, 
                alg='blr', 
                respfile=resp_file_te, 
                model_path=os.path.join(idp_dir,'Models'),
                outputsuffix=suffix)
    else:
        print('Estimating the normative model...')
        estimate(cov_file_tr, resp_file_tr, testresp=resp_file_te, 
                 testcov=cov_file_te, alg='blr', optimizer = 'l-bfgs-b', 
                 savemodel=True, warp=warp, warp_reparam=True)
        suffix = 'estimate'

Running IDP 0 FA_corpus_callosum :
Estimating the normative model...
Processing data in /home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/braincharts/models/test/FA_corpus_callosum/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running IDP 1 FA_uncinate_fasciculus :
Estimating the normative model...
Processing data in /home/preclineu/ramcir/Desktop/Diffusion/diffusion_nm/braincharts/models/test/FA_uncinate_fasciculus/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...


In [15]:
# initialise dataframe we will use to store quantitative metrics 
blr_metrics = pd.DataFrame(columns = ['eid', 'NLL', 'EV', 'MSLL', 'BIC','Skew','Kurtosis'])

for idp_num, idp in enumerate(idp_ids): 
    idp_dir = os.path.join(out_dir, idp)
    
    # load the predictions and true data. We use a custom function that ensures 2d arrays
    # equivalent to: y = np.loadtxt(filename); y = y[:, np.newaxis]
    yhat_te = load_2d(os.path.join(idp_dir, 'yhat_' + suffix + '.txt'))
    s2_te = load_2d(os.path.join(idp_dir, 'ys2_' + suffix + '.txt'))
    y_te = load_2d(os.path.join(idp_dir, 'resp_te.txt'))
    
    with open(os.path.join(idp_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle) 
    
    # compute error metrics
    if warp is None:
        metrics = evaluate(y_te, yhat_te)  
        
        # compute MSLL manually as a sanity check
        y_tr_mean = np.array( [[np.mean(y_tr)]] )
        y_tr_var = np.array( [[np.var(y_tr)]] )
        MSLL = compute_MSLL(y_te, yhat_te, s2_te, y_tr_mean, y_tr_var)         
    else:
        warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1] 
        W = nm.blr.warp
        
        # warp predictions
        med_te = W.warp_predictions(np.squeeze(yhat_te), np.squeeze(s2_te), warp_param)[0]
        med_te = med_te[:, np.newaxis]
       
        # evaluation metrics
        metrics = evaluate(y_te, med_te)
        
        # compute MSLL manually
        y_te_w = W.f(y_te, warp_param)
        y_tr_w = W.f(y_tr, warp_param)
        y_tr_mean = np.array( [[np.mean(y_tr_w)]] )
        y_tr_var = np.array( [[np.var(y_tr_w)]] )
        MSLL = compute_MSLL(y_te_w, yhat_te, s2_te, y_tr_mean, y_tr_var)     
    
    Z = np.loadtxt(os.path.join(idp_dir, 'Z_' + suffix + '.txt'))
    [skew, sdskew, kurtosis, sdkurtosis, semean, sesd] = calibration_descriptives(Z)
    
    BIC = len(nm.blr.hyp) * np.log(y_tr.shape[0]) + 2 * nm.neg_log_lik
    
    blr_metrics.loc[len(blr_metrics)] = [idp, nm.neg_log_lik, metrics['EXPV'][0], 
                                         MSLL[0], BIC, skew, kurtosis]
    
display(blr_metrics)

blr_metrics.to_pickle(os.path.join(out_dir,'blr_metrics.pkl'))

,eid,NLL,EV,MSLL,BIC,Skew,Kurtosis
0,FA_corpus_callosum,-37554.180136,0.092646,-17.47977,-75069.244948,-0.913438,4.800672
1,FA_uncinate_fasciculus,-32417.241847,0.026436,-0.01359,-64795.368370,-0.160598,0.772054


## Make predictions

In [16]:
site_ids_tr = df_tr.site.unique()
site_ids_te = df_te.site.unique()

In [19]:
display(X_te.shape)
display(X_tr.shape)

(4998, 14)

(17656, 14)

In [20]:
for idp_num, idp in enumerate(idp_ids): 
    print('Running IDP', idp_num, idp, ':')
    idp_dir = os.path.join(out_dir, idp)
    os.chdir(idp_dir)
    
    # extract and save the response variables for the test set
    y_te = df_te[idp].to_numpy()
    
    # save the variables
    resp_file_te = os.path.join(idp_dir, 'resp_te.txt') 
    np.savetxt(resp_file_te, y_te)
        
    # configure and save the design matrix
    cov_file_te = os.path.join(idp_dir, 'cov_bspline_te.txt')
    X_te = create_design_matrix(df_te[cols_cov], 
                                site_ids = df_te['site'],
                                all_sites = site_ids_te,
                                basis = 'bspline', 
                                xmin = xmin, 
                                xmax = xmax)
    np.savetxt(cov_file_te, X_te)
    
    # check whether all sites in the test set are represented in the training set
    if all(elem in site_ids_tr for elem in site_ids_te):
        print('All sites are present in the training data')
        
        # just make predictions
        yhat_te, s2_te, Z = predict(cov_file_te, 
                                    alg='blr', 
                                    respfile=resp_file_te, 
                                    model_path=os.path.join(idp_dir,'Models'))
    else:
        print('Some sites missing from the training data. Adapting model')
        
        # save the covariates for the adaptation data
        X_ad = create_design_matrix(df_ad[cols_cov], 
                                    site_ids = df_ad['site'],
                                    all_sites = site_ids_tr,
                                    basis = 'bspline', 
                                    xmin = xmin, 
                                    xmax = xmax)
        cov_file_ad = os.path.join(idp_dir, 'cov_bspline_ad.txt')          
        np.savetxt(cov_file_ad, X_ad)
        
        # save the responses for the adaptation data
        resp_file_ad = os.path.join(idp_dir, 'resp_ad.txt') 
        y_ad = df_ad[idp].to_numpy()
        np.savetxt(resp_file_ad, y_ad)
       
        # save the site ids for the adaptation data
        sitenum_file_ad = os.path.join(idp_dir, 'sitenum_ad.txt') 
        site_num_ad = df_ad['sitenum'].to_numpy(dtype=int)
        np.savetxt(sitenum_file_ad, site_num_ad)
        
        # save the site ids for the test data 
        sitenum_file_te = os.path.join(idp_dir, 'sitenum_te.txt')
        site_num_te = df_te['sitenum'].to_numpy(dtype=int)
        np.savetxt(sitenum_file_te, site_num_te)
         
        yhat_te, s2_te, Z = predict(cov_file_te, 
                                    alg = 'blr', 
                                    respfile = resp_file_te, 
                                    model_path = os.path.join(idp_dir,'Models'),
                                    adaptrespfile = resp_file_ad,
                                    adaptcovfile = cov_file_ad,
                                    adaptvargroupfile = sitenum_file_ad,
                                    testvargroupfile = sitenum_file_te)

Running IDP 0 FA_corpus_callosum :
All sites are present in the training data
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...
Running IDP 1 FA_uncinate_fasciculus :
All sites are present in the training data
Loading data ...
Prediction by model  1 of 1
Evaluating the model ...
Evaluations Writing outputs ...
Writing outputs ...
